#### SreeDananjay(21BAI1807)
#### **SLP Lab Assignment 4**

In [1]:
!pip install pandas scikit-learn gensim

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 3.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 3.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.0
    Uninstalling scipy-1.14.0:
      Successfully uninstalled scipy-1.14.0

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
import numpy as np

In [6]:
# Load the dataset
URL = "./SMSSpamCollection"
df = pd.read_csv(URL, sep='\t', header=None)
df.columns = ['label', 'sms_message']
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], df['label'], test_size=0.2, random_state=42)

In [8]:
# Vector Space Model (TF-IDF)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [16]:
model_tfidf = LogisticRegression()
model_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)
print(f'TF-IDF Accuracy: {accuracy_tfidf:.4f}')

TF-IDF Accuracy: 0.9749


In [10]:
# Word Embeddings (Word2Vec)
sentences = [sentence.split() for sentence in X_train]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

(605487, 695310)

In [11]:
def get_word2vec_features(sentences, model, vector_size):
    features = []
    for sentence in sentences:
        words = sentence.split()
        word_vectors = [model.wv[word] for word in words if word in model.wv]
        if word_vectors:
            features.append(np.mean(word_vectors, axis=0))
        else:
            features.append(np.zeros(vector_size))
    return np.array(features)

In [12]:
X_train_w2v = get_word2vec_features(X_train, word2vec_model, 100)
X_test_w2v = get_word2vec_features(X_test, word2vec_model, 100)

In [13]:
# Train and evaluate using Word2Vec
model_w2v = LogisticRegression()
model_w2v.fit(X_train_w2v, y_train)
y_pred_w2v = model_w2v.predict(X_test_w2v)
accuracy_w2v = accuracy_score(y_test, y_pred_w2v)
print(f'Word2Vec Accuracy: {accuracy_w2v:.4f}')

Word2Vec Accuracy: 0.9399
